In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import numpy as np
import pandas as pd
import gc

In [3]:
ncpu = 8 # It should be modified if run of midawy :)

In [4]:
store = pd.HDFStore('../Data/store1.h5')
print store

<class 'pandas.io.pytables.HDFStore'>
File path: ../Data/store1.h5
/miss             frame        (shape->[11437,3])    
/prop             frame        (shape->[2883630,254])
/train            frame        (shape->[90275,256])  


In [62]:
train = store["train"]
prop = store["prop"]

In [60]:
dd = ['type_architectural',
 'area_basement',
 'num_bathroom',
 'num_bedroom',
 'type_framing',
 'num_bathroom_calc',
 'type_deck',
 'area_liveperi_finished',
 'num_bath',
 'pooltypeid10',
 'region_county',
 'type_story',
 'type_material',
 'area_shed',
 'tax_year',
 'num_rot75_X',
 'num_rot75_Y']

train = train.drop(dd + ["area_total_calc"], axis=1)
prop = prop.drop(dd + ["area_total_calc"], axis=1)

print "prop shape " + str(prop.shape)
print "train shape " + str(train.shape)

prop shape (2883630, 236)
train shape (90275, 238)


In [64]:
important = ['cl3_num_bedroom', 'area_total_calc', 'cl1_num_bedroom_diff',
       'zip_tax_property_diff', 'num_rot30_X', 'city_num_bedroom_diff',
       'zon_build_year', 'cl3_tax_building_area_diff', 'tax_property_area',
       'neighbor_latitude_diff', 'neighbor_tax_property_area_diff',
       'cl3_num_bedroom_diff', 'num_pool', 'cluster3',
       'cl3_num_bathroom_diff', 'cl2_num_bedroom_diff',
       'region_city_count', 'neighbor_tax_building_diff',
       'cl2_tax_total_diff', 'cl1_tax_total_diff',
       'cl3_tax_property_area_diff', 'cl3_tax_total_diff',
       'zip_build_year', 'area_total_finished', 'cl2_area_lot_diff',
       'city_build_year_diff', 'zon_num_bedroom_diff',
       'cl2_area_total_calc_diff', 'city_tax_property_diff',
       'neighbor_tax_property_diff', 'city_tax_property_area_diff',
       'cl1_tax_property_area_diff', 'region_zip', 'cl1_area_lot_diff',
       'zon_longitude_diff', 'region_zip_count',
       'cl3_area_total_calc_diff', 'zip_area_lot_diff', 'zoning_property',
       'neighbor_longitude_diff', 'zon_tax_building_diff',
       'cl2_tax_building_diff', 'cluster5', 'tax_property',
       'zon_tax_building_area_diff', 'cl3_build_year', 'cluster4_count',
       'cl1_tax_building_area_diff', 'zip_tax_property_area_diff',
       'city_tax_total_diff', 'num_rot45_X', 'cl1_area_total_calc_diff',
       'cl1_tax_building_diff', 'tax_total', 'city_longitude_diff',
       'cl3_tax_building_diff', 'city_tax_building_diff', 'cluster4',
       'zon_tax_property_diff', 'city_area_total_calc_diff',
       'cl2_tax_property_area_diff', 'zip_tax_total_diff',
       'zon_tax_property_area_diff', 'zip_longitude_diff',
       'neighbor_tax_total_diff', 'zon_build_year_diff',
       'zon_latitude_diff', 'cl1_build_year_diff', 'tax_building',
       'cl3_tax_property_diff', 'neighbor_build_year_diff',
       'cl3_area_lot_diff', 'cluster5_count', 'zip_num_bedroom_diff',
       'zon_tax_total_diff', 'neighbor_area_lot_diff',
       'zip_tax_building_diff', 'zip_build_year_diff', 'zip_latitude_diff',
       'neighbor_area_total_calc_diff', 'cl3_build_year_diff',
       'cl1_latitude_diff', 'area_lot', 'cl2_longitude_diff',
       'cl1_tax_property_diff', 'city_latitude_diff', 'city_area_lot_diff',
       'build_year', 'cl2_latitude_diff', 'area_ratio',
       'cl3_latitude_diff', 'zip_area_total_calc_diff',
       'cl2_tax_property_diff', 'cl2_build_year_diff',
       'cl3_longitude_diff', 'cl1_longitude_diff', 'zon_area_lot_diff',
       'tax_land', 'area_live_finished', 'zon_area_total_calc_diff']

train = train[important + ["id_parcel", "logerror", "month"]]
prop = prop[important + ["id_parcel"]]

print "prop shape " + str(prop.shape)
print "train shape " + str(train.shape)

prop shape (2883630, 101)
train shape (90275, 103)


In [65]:
for col in prop.columns:
    prop[col]=prop[col].fillna(prop[col].median())
    train[col]=train[col].fillna(prop[col].median())

In [66]:
y = train.logerror
mid = np.percentile(y, 50)
y = y - mid
q1 = np.percentile(y, 25)
q3 = np.percentile(y, 75)
print q1, q3
interval = q3 - q1
fac = 8.0
interval = interval * fac / 2.
hi = interval + mid
lo = -interval + mid
print hi, lo

-0.0313 0.0332
0.264 -0.252


In [67]:
# split the data to 9 months for train and 3 months for test
x1 = train[train.month < 10]    # use for train
x0 = train[train.month > 9]     # use for test
print "Size of the x1 data frame: ", x1.shape
print "Size of the x0 data frame: ", x0.shape

y1 = x1['logerror'].values
y0 = x0['logerror'].values

index_hi = y1 > hi   # drop 1480 points
index_lo = y1 < lo    # drop 947 points
print sum(index_hi), sum(index_lo)

y1 = y1[(~index_lo) & (~index_hi)]
x1 = x1[(~index_lo) & (~index_hi)]

print "Size of the x1 data frame: ", x1.shape
print "Size of the x0 data frame: ", x0.shape

Size of the x1 data frame:  (81733, 103)
Size of the x0 data frame:  (8542, 103)
2088 1432
Size of the x1 data frame:  (78213, 103)
Size of the x0 data frame:  (8542, 103)


In [99]:
np.random.seed(123)
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.recurrent import LSTM
# define custom R2 metrics for Keras backend
from keras import backend as K
# to tune the NN
from keras.constraints import maxnorm
from keras.optimizers import SGD, Adam

In [100]:
input_dims = x1.shape[1] - 3

In [101]:
def NN_model():
    model = Sequential()
    # Input layer with dimension input_dims and hidden layer i with input_dims neurons. 
    model.add(Dense(input_dims, input_dim=input_dims, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Activation("linear"))

    # Hidden layer
    model.add(Dense(input_dims//2, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Activation("linear"))
    # Hidden layer
    model.add(Dense(input_dims//4, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Activation("linear"))
    
    # Output Layer.
    model.add(Dense(1))
    # Use a large learning rate with decay and a large momentum. 
    # compile this model
    model.compile(loss='mean_absolute_error', #'mean_squared_error', # one may use 'mean_absolute_error' as alternative
                  optimizer='rmsprop')
    
    # Visualize NN architecture
#    print(model.summary())
    return model

In [102]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=NN_model, nb_epoch=300, batch_size=30, verbose=0)))
model = Pipeline(estimators)

In [103]:
from sklearn.metrics import mean_absolute_error

In [106]:
model.fit(x1.drop(["id_parcel", "month", "logerror"], axis=1), y1) # Train the model without outliers

print "Error on training data ", mean_absolute_error(y1, model.predict(x1.drop(["id_parcel", "month", "logerror"], axis=1)))
print "Error on 3 months test ", mean_absolute_error(y0, model.predict(x0.drop(["id_parcel", "month", "logerror"], axis=1)))

Error on training data  0.0460231832904
Error on 3 months test  0.0656488194521


# CV

In [107]:
# we have duplicate in train :(
# we can have three simple strategies
# 1- keep first one; 2- keep last one; 3- average
# I think the logerror reduce from first to last but I am not sure
# it is a very important point
duplicate = train["id_parcel"].duplicated(keep='first')
train = train[~duplicate]
print "Size of the train data frame: ", train.shape
print "Size of the prop data frame: ", prop.shape

Size of the train data frame:  (90150, 103)
Size of the prop data frame:  (2883630, 101)


In [108]:
y = train.logerror
mid = np.percentile(y, 50)
y = y - mid
q1 = np.percentile(y, 25)
q3 = np.percentile(y, 75)
print q1, q3

fac = 8.0
interval = q3 - q1
interval = interval * fac / 2.
hi_train = interval + mid
lo_train = -interval + mid

fac = 65.0
interval = q3 - q1
interval = interval * fac / 2.
hi_test = interval + mid
lo_test = -interval + mid

print lo_train, hi_train
print lo_test, hi_test

-0.0313 0.0332
-0.252 0.264
-2.09025 2.10225


In [109]:
y = train['logerror'].values
x = train.drop(['month', 'logerror'], axis=1)
print "Size of the train data frame: ", x.shape
print "Size of the prop  data frame: ", prop.shape

print("Generate a list of outliers should be droped for training")
index_hi = y > hi_train
index_lo = y < lo_train
print sum(index_hi), sum(index_lo)

outliers_train = []
for ii in range(y.shape[0]):
    if index_hi[ii] or index_lo[ii]:
        outliers_train.append(ii)

print("Generate a list of outliers should be droped for testing")
index_hi = y > hi_test
index_lo = y < lo_test
print sum(index_hi), sum(index_lo)

outliers_test = []
for ii in range(y.shape[0]):
    if index_hi[ii] or index_lo[ii]:
        outliers_test.append(ii)

Size of the train data frame:  (90150, 101)
Size of the prop  data frame:  (2883630, 101)
Generate a list of outliers should be droped for training
2310 1568
Generate a list of outliers should be droped for testing
51 46


In [110]:
def splitDataFrameIntoSmaller(df, chunkSize = 100000):
    listOfDf = list()
    numberChunks = len(df) // chunkSize + 1
    for i in range(numberChunks):
        listOfDf.append(i*chunkSize)
    listOfDf.append(len(df))
    return listOfDf

split_index = splitDataFrameIntoSmaller(prop)

In [58]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 44)

train_pred = np.zeros(train.shape[0], dtype=np.float16)
prop_pred = np.zeros(prop.shape[0], dtype=np.float16)
scores1 = []; scores2 = []

for _ in range(10):
    for train_index, test_index in skf.split(x, y):

        train_index_wo = [ix for ix in train_index if ix not in outliers_train]
        test_index_wo = [ix for ix in test_index if ix not in outliers_test]

        x1, x0 = x.iloc[train_index_wo], x.iloc[test_index_wo]
        y1, y0 = y[train_index_wo], y[test_index_wo]

        model.fit(x1.drop(["id_parcel"], axis=1), y1) # Train the model without outliers

        #calculate score without second outliers
        scores1.append(mean_absolute_error(y0, model.predict(x0.drop(["id_parcel"], axis=1))))
        print "Score without outliers for the ", len(scores1), " fold is ", scores1[len(scores1)-1]

        #calculate score with outliers
        x0 = x.iloc[test_index]
        y0 = y[test_index]

        pred = model.predict(x0.drop(["id_parcel"], axis=1))
        scores2.append(mean_absolute_error(y0, pred))
    #    print "Score with outliers for the ", len(scores2), " fold is ", scores2[len(scores2)-1]

        for ii, idx in enumerate(test_index):
            train_pred[idx] = pred[ii]

        for ii in range(0, len(split_index)-1):
            n1 = split_index[ii]; n2 = split_index[ii+1]
            pred = model.predict(prop.iloc[n1:n2].drop(['id_parcel'], axis=1))
            prop_pred[n1:n2] += pred

print "Average score without outliers over all folds : " , np.mean(scores1), " ", np.std(scores1)
print "Average score with    outliers over all folds : " , np.mean(scores2), " ", np.std(scores2)

Score without outliers for the  1  fold is  0.0676785859112


KeyboardInterrupt: 

In [ ]:
out = pd.DataFrame()
out["ParcelId"] = prop["id_parcel"]
months = ["201610", "201611", "201612", "201710", "201711", "201712"]
for col in months:
    out[col] = map(lambda x: x/10.0, prop_pred)
    
out_train = pd.DataFrame()
out_train["ParcelId"] = train["id_parcel"]
for col in months:
    out_train[col] = train_pred #+ 0.02 #IMPORTANT POINT: I add a constant to train prediction


print("Read the missing")
miss = store["miss"]

med = train.logerror.median()
for col in months:
    miss[col] = med
    
miss = miss[["id_parcel"]+months]
miss.columns = ["ParcelId"] + months

out = pd.concat([out, out_train, miss], axis=0)

from datetime import datetime
out.to_csv('rf.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False, float_format='%.4f')